# Schema Matching

Suppose we are given schemas $S = \left \{s_0, s_1, s_2, \ldots s_n \right \}$ and $T = \left \{t_0, t_1, t_2, \ldots t_m \right \}$. 

> *Note:* $S$ and $T$ refer to the set of elements (attributes) in the schemas.

The elements of $S$ can be enumerated as: $$s_{i} \in S \quad i \in \left \{0, 1, 2, \ldots n \right \}$$

The same can be done for the elements of $T$: $$t_{j} \in T \quad  j \in \left \{ 0, 1, 2, \ldots m \right \}$$

The **goal** in **schema matching** is to find the elements in $S$ and $T$ which are a match, 
that is, find $s_i$ and $t_j$ s.t. $\left (s_i, t_j \right ) = \text{match}$. This is done by creating a **matcher** which takes us input schemas $S$ and $T$ and outputs a **similarity matrix** which holds a **similarity score** $\forall \left (s_i, t_j \right )$ pairs.

<br> We can then establish some **threshold** $t$ so that $(s_i, t_j) = \text{match}$ i.f.f $simscore(s_i, t_j)$ >= $t$

<br>

$$\longrightarrow \texttt{matcher}(S,T) = \texttt{Similarity Matrix}$$

<br> <br> 
\begin{gather*}
\texttt{Similarity Matrix} =
\begin{bmatrix}
\texttt{sim}_{00} & \texttt{sim}_{01} & \ldots & \texttt{sim}_{0m}\\ 
\texttt{sim}_{10} & \texttt{sim}_{11} &  \ldots & \texttt{sim}_{1m}\\ 
\vdots & \vdots &  \ddots & \vdots \\
\texttt{sim}_{n0} & \texttt{sim}_{n1} & \ldots & \texttt{sim}_{nm}
\end{bmatrix} \quad \text{where} & \texttt{sim}_{ij} = \texttt{sim score}(s_{i}, t_{j})
\end{gather*}

In [107]:
import numpy as np
import pandas as pd 

## K-grams

In [108]:
def K_grams(X, k):
    X = X.center((len(X)+2)) # add space at the begining and end of the string
    kgrams = []
    for i, x in enumerate(X):
        if len(X[i:]) >= k:
            kgrams.append(X[i:i+k])
    return(kgrams)

## Jaccard Similarity 

In [109]:
def jaccard_sim(X, Y, k):
    X_k = set(K_grams(X, k))
    Y_k = set(K_grams(Y, k))
    XinterY = X_k.intersection(Y_k)
    XunionY = X_k.union(Y_k)
    return(len(XinterY )/len(XunionY))

<br> For this schema matching example we will be using two different types of matchers: **name-based** and **instance-based**. We will be using three different datasets. The first dataset will be called `chicago` and contains demographic information on the different neighborhoods in Chicago. Its schema is: 

<br> `{'CommunityAreaNumber', 'CommunityAreaName', 'PercentofHousingCrowded', 'PercentHouseholdsBelowPoverty', 'PercentAged16OverUnemployed', 'PercentAged25PlusWithoutHighSchoolDiploma', 'PercentAgedUnder18orOver64 ', 'PerCapitaIncome', 'HardshipIndex'}`

<br> The second dataset will be called `zipcodes` and it contains zipcodes for every neighborhood in Chicago. Its schema is:

<br> `{'CommunityArea', 'Zipcode'}`

<br> The third dataset will be called `schools` and it contains some basic information about the public schools in Chicago like geographical location, level of attendance, test scores, etc. Its schema is:

<br> `{'School ID', 'Name of School', 'Street Address', 'City', 'State', 'Zipcode', 'Phone Number', , 'NWEA Reading Growth Percentile All Grades', ..., 'NWEA Reading Growth Percentile Grade 3', 'Suspensions Per 100 students 2013','One-Year Drop Out Rate Percentage 2013',  'X Coordinate', 'Y Coordinate', 'Longitude', 'Latitude', 'Location'}`

<br> *Note:* This example is to illustrate the concept of schema matching so it is obvious what the matches should be; however, in practice schemas will be far more complex and as a result more complex matchers will be needed.


#### Reading in the datasets:

In [110]:
chicago = pd.read_csv("data/ChicagoDataPortal.csv")
chicago.columns = ['CommunityAreaNumber', 'CommunityAreaName',
       'PercentofHousingCrowded', 'PercentHouseholdsBelowPoverty',
       'PercentAged16OverUnemployed',
       'PercentAged25PlusWithoutHighSchoolDiploma',
       'PercentAgedUnder18orOver64 ', 'PerCapitaIncome', 'HardshipIndex']
chicago.head()

,CommunityAreaNumber,CommunityAreaName,PercentofHousingCrowded,PercentHouseholdsBelowPoverty,PercentAged16OverUnemployed,PercentAged25PlusWithoutHighSchoolDiploma,PercentAgedUnder18orOver64,PerCapitaIncome,HardshipIndex
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939.0,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040.0,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787.0,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524.0,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123.0,6.0


In [111]:
zipcodes = pd.read_csv("data/Zipcodes.csv", header=None)
zipcodes.columns = ['CommunityArea', 'Zipcode']
zipcodes.head()

,CommunityArea,Zipcode
0,Cathedral District,60611
1,Central Station,60605
2,Dearborn Park,60605
3,Gold Coast,60610
4,Gold Coast,60611


In [112]:
schools = pd.read_csv("data/chicagopublicschools2014.csv")
schools.head()

,School ID,Name of School,Street Address,City,State,Zipcode,Phone Number,Website,Blue Ribbon Award,CPS Performance Policy Level,...,Teacher Attendance Percentage 2013,Teacher Attendance Percentage 2012,Grade 3-8 On-Track Percentage 2013,One-Year Drop Out Rate Percentage 2013,One-Year Drop Out Rate Percentage 2012,X Coordinate,Y Coordinate,Longitude,Latitude,Location
0,400048,LEARN Charter ES - Excel,2401 W Congress Pkwy,Chicago,IL,60612,(312) 243-7001,http://www.learncharter.org,NaN,LEVEL 2,...,NaN,NaN,NaN,NaN,NaN,1160463.303,1897662.506,41.874886,-87.686311,"{'latitude': '-87.68631142', 'longitude': '41...."
1,609871,Barbara Vick Early Childhood & Family Center,2554 W 113th St,Chicago,IL,60655,(773) 535-2671,http://www.barbaravick.cps.k12.il.us,NaN,NaN,...,94.8,95.6,NaN,NaN,NaN,1161152.120,1829643.669,41.688218,-87.685663,"{'latitude': '-87.6856634', 'longitude': '41.6..."
2,610083,Daniel C Beard Elementary School,6445 W Strong St,Chicago,IL,60656,(773) 534-1228,http://www.cps.edu,NaN,NaN,...,95.2,95.4,67.6,NaN,NaN,1132103.024,1932162.514,41.970097,-87.789637,"{'latitude': '-87.78963691', 'longitude': '41...."
3,610066,James E McDade Elementary Classical School,8801 S Indiana Ave,Chicago,IL,60619,(773) 535-3669,http://www.mcdadeclassicalschool.org/,NaN,LEVEL 1,...,93.1,96.2,94.6,NaN,NaN,1179172.217,1846656.745,41.734514,-87.619177,"{'latitude': '-87.61917677', 'longitude': '41...."
4,400092,Chicago Intl Charter - Lloyd Bond,13300 S Langley Ave,Chicago,IL,60827,(773) 468-1300,http://www.chicagointl.org/,NaN,LEVEL 3,...,NaN,NaN,NaN,NaN,NaN,1183271.540,1817218.320,41.653637,-87.605071,"{'latitude': '-87.60507067', 'longitude': '41...."


## Name-based Matchers 

<br> Name-based matchers use the **names** of the elements in each schema to determine a match. Each name is a string so we can use a string similarity metric to calculate the *similarity score*.

<br> Thus for name-based matchers we have the following setup, <br> <br>

$$\forall s_i \in S \quad s_i = \text{string}$$ and  $$\forall t_j \in T \quad t_j = \text{string}$$

$$sim score(s_i, t_j) = JaccardSimilarity(s_{ik}, t_{jk}) = \frac{\left | s_{ik} \cap t_{jk} \right |}{\left | s_{ik} \cup t_{jk} \right |}$$

> $s_{ik}$ and $t_{jk}$ refer to the k-gram (also known as n-gram) sets of the respective elements. For more information on k-grams and string similarity you can refer to this earlier notebook on string similarity algorithms


In [113]:
def name_based(S, T, k=8):
    n = len(S.columns)
    m = len(T.columns)
    Sim = np.zeros((n,m))
    s_i = []
    t_j = []

    for i in range(n):
        for j in range(m):
            s_i.append(S.columns[i])
            t_j.append(T.columns[j])
            score = jaccard_sim(S.columns[i], T.columns[j], k=k)
            Sim[i, j] = score
    return(Sim, s_i, t_j)

In [114]:
""" function to retrive the similarity matrix of a matcher and format
the results into an easily readable dataframe """

def matcher_results(matcher, S, T, threshold=0.3):
    Sim, s_i, t_j = matcher(S, T)
    sim_scores = Sim.ravel()
    match = np.where(sim_scores > threshold, 1, 0)
    data = {'s_i': s_i, 't_j': t_j, 'sim score': sim_scores, 'match':match}
    df = pd.DataFrame(data, columns = data.keys())
    return Sim, df

Let $S$ = `chicago` and $T$ = `zipcodes`

In [115]:
Sim, df = matcher_results(name_based, S=chicago, T=zipcodes)
Sim

array([[0.46666667, 0.        ],
       [0.53846154, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [116]:
df.head()

,s_i,t_j,sim score,match
0,CommunityAreaNumber,CommunityArea,0.466667,1
1,CommunityAreaNumber,Zipcode,0.000000,0
2,CommunityAreaName,CommunityArea,0.538462,1
3,CommunityAreaName,Zipcode,0.000000,0
4,PercentofHousingCrowded,CommunityArea,0.000000,0


In [117]:
df[df.match == 1]

,s_i,t_j,sim score,match
0,CommunityAreaNumber,CommunityArea,0.466667,1
2,CommunityAreaName,CommunityArea,0.538462,1


Let $S$ = `zipcodes` and $T$ = `schools`

In [118]:
Sim, df = matcher_results(name_based, S=zipcodes, T=schools)
Sim

array([[0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.1875, 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ,

In [119]:
df.head()

,s_i,t_j,sim score,match
0,CommunityArea,School ID,0.0,0
1,CommunityArea,Name of School,0.0,0
2,CommunityArea,Street Address,0.0,0
3,CommunityArea,City,0.0,0
4,CommunityArea,State,0.0,0


In [120]:
df[df.match == 1]

,s_i,t_j,sim score,match
79,Zipcode,Zipcode,1.0,1


In [121]:
pd.isnull(chicago).sum()
chicago = chicago.drop(pd.isnull(chicago).any(1).nonzero()[0])
#pd.isnull(zipcodes).sum()
#pd.isnull(schools).sum()
col_to_drop = np.where(pd.isnull(schools).sum() > 40)[0]
schools = schools.drop(schools.columns[col_to_drop], axis=1)
schools = schools.drop(pd.isnull(schools).any(1).nonzero()[0])

In [122]:
chicago = chicago.astype(str)
zipcodes = zipcodes.astype(str)
schools = schools.astype(str)

##  Instance-based Matchers

<br> Instance-based matchers determine the similarity between the sets of **data instances** of each element. Then we can measure the similarity of the data instances by calculating the **overlap** of the sets.

<br> For instance-based matchers we have the following setup, 

<br> Let $\left | s_i \right | = p$ and $\left | t_j \right | = q$

<br> $$\forall s_i \in S$$
<br> $$\quad s_i = \left \{ d_{ik} \right \} \quad k \in \left \{0, 1, 2, \ldots p \right \}$$
<br> $$\left \{ d_{ik} \right \} = \left \{ d_0, d_1, d_2, \ldots, d_p \right \} $$


<br> $$\forall t_j \in T$$
<br> $$\quad t_j = \left \{ d_{jl} \right \} \quad l \in \left \{0, 1, 2, \ldots q \right \}$$
<br> $$\left \{ d_{jl} \right \} = \left \{ d_0, d_1, d_2, \ldots, d_q \right \} $$


* $p$ is the number of data instances in $s_i$
* $q$ is the number of data instances in $t_j$
* $d_{ik}$ is the set of data instances for $s_i$ where $k$ is the index on $s_i$
* $d_{jl}$ is the set of data instances for $t_j$ where $l$ is the index on $t_j$

<br> $$sim score(s_i, t_j) = Overlap(d_{ik}, d_{jl}) = \frac{\left | d_{ik} \cap d_{jl} \right |}{ min \left ( \left | d_{ik} \right |, \left | d_{jl} \right | \right )}$$



In [123]:
def overlap_measure(d_i, d_j):
    inter = d_i.intersection(d_j)
    cardinality_inter = len(inter)
    overlap = cardinality_inter/min(len(d_i), len(d_j))
    return(overlap)

In [124]:
def instance_based(S, T):
    s_i_order = []
    t_j_order = []
    n = len(S.columns)
    m = len(T.columns)
    Sim = np.zeros((n,m))

    for i in range(n):
        for j in range(m):
            s_i_order.append(S.columns[i])
            t_j_order.append(T.columns[j])
            d_i = set(S[S.columns[i]]) # data instances of elem s_i
            d_j = set(T[T.columns[j]]) # data instances of elem t_j
            overlap = overlap_measure(d_i, d_j)
            Sim[i, j] = overlap

    return(Sim, s_i_order, t_j_order)  

Let $S$ = `chicago` and $T$ = `zipcodes`

In [125]:
Sim, df = matcher_results(instance_based, S=chicago, T=zipcodes)
Sim

array([[0.        , 0.        ],
       [0.36363636, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [126]:
df.head()

,s_i,t_j,sim score,match
0,CommunityAreaNumber,CommunityArea,0.000000,0
1,CommunityAreaNumber,Zipcode,0.000000,0
2,CommunityAreaName,CommunityArea,0.363636,1
3,CommunityAreaName,Zipcode,0.000000,0
4,PercentofHousingCrowded,CommunityArea,0.000000,0


In [127]:
df[df.match == 1]

,s_i,t_j,sim score,match
2,CommunityAreaName,CommunityArea,0.363636,1


Let $S$ = `zipcodes` and $T$ = `schools`

In [128]:
Sim, df = matcher_results(instance_based, S=zipcodes, T=schools)
Sim

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.76, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ]])

In [129]:
df.head()

,s_i,t_j,sim score,match
0,CommunityArea,School ID,0.0,0
1,CommunityArea,Name of School,0.0,0
2,CommunityArea,Street Address,0.0,0
3,CommunityArea,City,0.0,0
4,CommunityArea,State,0.0,0


In [130]:
df[df.match == 1]

,s_i,t_j,sim score,match
39,Zipcode,Zipcode,0.76,1


## <font color=k> Combining Matchers </font>

#### <font color=k> $$\mathit{combined(i, j)} = \bigg[ \sum_{m=1}^{k} \mathit{matcherScore}(m, i, j) \bigg] / k$$ </font>

* $\mathit{matcherScore(m, i, j)}$ is the similarity score between $s_i$ and $t_j$ as produced by the $m$<sup>th</sup> matcher


In [131]:
matchers = [name_based, instance_based]

def combined(S, T, matchers):
    k = len(matchers)
    n = len(S.columns)
    m = len(T.columns) # m here is the number of elements in T
    matcher_scores = np.zeros((n*m,1))
    
    for m in range(k): # now m is the number of matchers 
        scores = matchers[m](S, T)[0].ravel()
        matcher_scores = np.append(matcher_scores, scores[:, np.newaxis], axis=1)
        combined_scores = np.sum(matcher_scores, axis=1)/k
        
    return(combined_scores)

Let $S$ = `chicago` and $T$ = `zipcodes`

In [132]:
""" 
function to retrieve the corresponding s_i, t_j elements for each score
since combined() did not store it
""" 
def order(S,T):
    s_i = []
    t_j = []
    for i in range(len(S.columns)):
        for j in range(len(T.columns)):
            s_i.append(S.columns[i])
            t_j.append(T.columns[j])
    return(s_i, t_j)

def combined_results(S, T, matchers, threshold=0.4):
    combined_scores = combined(S, T, matchers).ravel()
    match = np.where(combined_scores > threshold, 1, 0)
    s_i, t_j = order(S, T)
    data = {'s_i': s_i, 't_j': t_j, 'combined score': combined_scores, 'match': match}
    df = pd.DataFrame(data, columns = data.keys())
    return df

In [133]:
df = combined_results(S=chicago, T=zipcodes, matchers=matchers)
df.head()

,s_i,t_j,combined score,match
0,CommunityAreaNumber,CommunityArea,0.233333,0
1,CommunityAreaNumber,Zipcode,0.000000,0
2,CommunityAreaName,CommunityArea,0.451049,1
3,CommunityAreaName,Zipcode,0.000000,0
4,PercentofHousingCrowded,CommunityArea,0.000000,0


In [134]:
df[df.match == 1]

,s_i,t_j,combined score,match
2,CommunityAreaName,CommunityArea,0.451049,1


In [135]:
df = combined_results(S=zipcodes, T=schools, matchers=matchers)
df.head()

,s_i,t_j,combined score,match
0,CommunityArea,School ID,0.0,0
1,CommunityArea,Name of School,0.0,0
2,CommunityArea,Street Address,0.0,0
3,CommunityArea,City,0.0,0
4,CommunityArea,State,0.0,0


In [136]:
df[df.match == 1]

,s_i,t_j,combined score,match
39,Zipcode,Zipcode,0.88,1
